<a href="https://colab.research.google.com/github/gaurav22m/Braille-to-English-Converter/blob/main/braille_vg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import sklearn
from tensorflow import keras
import os
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import ToTensor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Function to load images from a folder
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

# Function to preprocess images
def preprocess_images(images):
    # Resize images to a consistent size
    target_size = (100, 100)  # Specify the target size
    resized_images = resize_images(images, target_size)

    # Convert images to grayscale
    grayscale_images = convert_to_grayscale(resized_images)

    # Normalize pixel values
    normalized_images = normalize_images(grayscale_images)

    # Augment dataset if necessary
    augmented_images = augment_dataset(normalized_images)

    return augmented_images

# Function to resize images
def resize_images(images, target_size):
    resized_images = []
    for image in images:
        resized_image = cv2.resize(image, target_size)
        resized_images.append(resized_image)
    return resized_images

# Function to convert images to grayscale
def convert_to_grayscale(images):
    grayscale_images = []
    for image in images:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        grayscale_images.append(grayscale_image)
    return grayscale_images

# Function to normalize pixel values
def normalize_images(images):
    normalized_images = []
    for image in images:
        normalized_image = image / 255.0  # Normalize pixel values to range [0, 1]
        normalized_images.append(normalized_image)
    return normalized_images

# Function to augment dataset
# Transformations for scaling and adding noise
def augment_dataset(images):
    augmented_images = []
    scale_transform = transforms.Resize((int(1.5 * images[0].shape[0]), int(1.5 * images[0].shape[1])))
    to_tensor_transform = transforms.ToTensor()  # Convert PIL image to tensor
    noise_transform = transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.1)  # Add Gaussian noise

    for image in images:
        # Convert NumPy array to PIL image
        pil_image = Image.fromarray(image)
        # Apply transformations
        scaled_image = scale_transform(pil_image)
        tensor_image = to_tensor_transform(scaled_image)  # Convert PIL image to tensor
        noisy_image = noise_transform(tensor_image)
        # Convert back to NumPy array
        noisy_image_np = noisy_image.permute(1, 2, 0).numpy()  # Convert tensor to NumPy array
        # Append the augmented image to the list
        augmented_images.append(noisy_image_np)

    return augmented_images

# Path to the folder containing images
folder_path = '/content/drive/MyDrive/Braille Dataset2'

# Load images from each class folder
images_per_class = {}
for class_folder in os.listdir(folder_path):
    class_path = os.path.join(folder_path, class_folder)
    if os.path.isdir(class_path):
        images = load_images_from_folder(class_path)
        preprocessed_images = preprocess_images(images)
        images_per_class[class_folder] = preprocessed_images

# Now 'images_per_class' dictionary contains preprocessed images for each class

# Function to save preprocessed images to disk
def save_preprocessed_images(images, output_folder):
    for idx, image in enumerate(images):
        output_path = os.path.join(output_folder, f"image_{idx}.png")
        cv2.imwrite(output_path, (image * 255).astype(np.uint8))

# Path to the folder to save preprocessed images
output_folder = '/content/drive/MyDrive/Braille Dataset preprocessed'

# Save preprocessed images for each class
for class_name, images in images_per_class.items():
    class_output_folder = os.path.join(output_folder, class_name)
    os.makedirs(class_output_folder, exist_ok=True)
    save_preprocessed_images(images, class_output_folder)


In [ ]:
from sklearn.model_selection import train_test_split

# Function to split dataset into train, validation, and test sets
def split_dataset(images_per_class, test_size=0.2, validation_size=0.25, random_state=None):
    train_images = []
    train_labels = []
    validation_images = []
    validation_labels = []
    test_images = []
    test_labels = []

    for class_name, images in images_per_class.items():
        # Split images for each class
        X_train, X_test, y_train, y_test = train_test_split(images, [class_name]*len(images), test_size=test_size, random_state=random_state, stratify=[class_name]*len(images))
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size/(1-test_size), random_state=random_state, stratify=y_train)

        # Add split images to respective sets
        train_images.extend(X_train)
        train_labels.extend(y_train)
        validation_images.extend(X_val)
        validation_labels.extend(y_val)
        test_images.extend(X_test)
        test_labels.extend(y_test)

    return train_images, validation_images, test_images, train_labels, validation_labels, test_labels

# Split dataset
train_images, validation_images, test_images, train_labels, validation_labels, test_labels = split_dataset(images_per_class)

# Print sizes of each set
print("Train set size:", len(train_images))
print("Validation set size:", len(validation_images))
print("Test set size:", len(test_images))

Train set size: 858
Validation set size: 390
Test set size: 312


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define VGG16 architecture
class VGG16(nn.Module):
    def __init__(self, num_classes=26):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),  # Change input channels to 1
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Define hyperparameters
learning_rate = 0.001
batch_size = 64
num_epochs = 10

# Initialize model
model = VGG16()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define batch size and number of epochs
batch_size = 64
num_epochs = 10

# Define the path to the train folder
train_folder_path = '/content/drive/MyDrive/Splitted_braille/train'

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.Grayscale(num_output_channels=1),  # Convert images to grayscale (1 channel)
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.485], std=[0.229])  # Normalize with ImageNet mean and std for grayscale images
])

# Load the dataset
train_dataset = datasets.ImageFolder(root=train_folder_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


validation_folder_path = '/content/drive/MyDrive/Splitted_braille/validation'
if os.path.exists(validation_folder_path):
    validation_dataset = datasets.ImageFolder(root=validation_folder_path, transform=transform)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
else:
    validation_loader = None

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

# Validation
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
  for inputs, labels in validation_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()  # Fix: call .sum().item() to get the total number of correct predictions
      loss = criterion(outputs, labels)
      val_loss += loss.item() * inputs.size(0)

  # Print statistics
  epoch_loss = running_loss / len(train_loader.dataset)
  epoch_val_loss = val_loss / len(validation_loader.dataset)
  accuracy = correct / total
  print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}, Accuracy: {accuracy:.2%}")

print('Finished Training')

In [ ]:
# Set model to evaluation mode
model.eval()

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the validation set and make predictions
with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs = inputs.double()  # Convert inputs to double
        labels = torch.tensor(labels)  # Convert labels to tensor format
        outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.numpy())
        predicted_labels.extend(predicted.numpy())

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
Initialize empty lists to store precision values and epochs
validation_precision = []
epochs = range(1, num_epochs + 1)

# Validation loop
for epoch in range(num_epochs):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.double()  # Convert inputs to double
            labels = torch.tensor(labels)  # Convert labels to tensor format
            outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate precision for this epoch
        precision = correct / total
        validation_precision.append(precision)

# Plot validation precision
plt.figure(figsize=(10, 6))
plt.plot(epochs, validation_precision, marker='o', color='blue')
plt.title('Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.xticks(epochs)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Initialize empty lists to store precision and recall values for each epoch
validation_precision = []
validation_recall = []
epochs = range(1, num_epochs + 1)

# Validation loop
for epoch in range(num_epochs):
    model.eval()
    correct = 0
    total = 0
    true_positives = 0
    false_negatives = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.double()  # Convert inputs to double
            labels = torch.tensor(labels)  # Convert labels to tensor format
            outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            true_positives += ((predicted == labels) & (labels == 1)).sum().item()
            false_negatives += ((predicted != labels) & (labels == 1)).sum().item()

        # Calculate precision and recall for this epoch
        precision = true_positives / (true_positives + (total - true_positives - false_negatives))
        recall = true_positives / (true_positives + false_negatives)
        validation_precision.append(precision)
        validation_recall.append(recall)

# Plot validation precision
plt.figure(figsize=(10, 6))
plt.plot(epochs, validation_precision, marker='o', color='blue')
plt.title('Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.xticks(epochs)
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot validation recall
plt.figure(figsize=(10, 6))
plt.plot(epochs, validation_recall, marker='o', color='green')
plt.title('Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.xticks(epochs)
plt.grid(True)
plt.tight_layout()
plt.show()